In [ ]:
import os
import string
import unicodedata
from typing import Any, Dict, Iterator, List, Tuple, Union
import random
import pandas as pd

import datasets
import numpy as np
from pytorch_lightning import LightningModule, Trainer, seed_everything
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim import lr_scheduler
from transformers import BertJapaneseTokenizer,ElectraTokenizer,T5Tokenizer
import transformers 
from transformers import RobertaTokenizer, RobertaModel
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import callbacks
from pytorch_lightning import loggers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AdamW
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
args = {
    'random_seed': 42,  # Random Seed
    # Transformers PLM name.
    'pretrained_model': 'cl-tohoku/bert-base-japanese-whole-word-masking',
    # Optional, Transformers Tokenizer name. Overrides `pretrained_model`
    'pretrained_tokenizer': 'cl-tohoku/bert-base-japanese-whole-word-masking',
    'norm_form': 'NFKC',
    'batch_size': 8,  # <=32 for TPUv2-8
    'lr': 2e-5,  # Learning Rate
    'max_length': 384,  # Max Length input size
    'doc_stride': 128,  # The interval of the context when splitting is needed
    'epochs': 3,  # Max Epochs
    'dataset': 'SkelterLabsInc/JaQuAD',
    'huggingface_auth_token': None,
    'test_mode': False,  # Test Mode enables `fast_dev_run`
    'optimizer': 'AdamW',
    'weight_decay': 0.01,  # Weight decaying parameter for AdamW
    'lr_scheduler': 'warmup_lin',
    'warmup_ratio': 0.1,
    'fp16': False,  # Enable train on FP16 (if GPU)
    'tpu_cores': 8,  # Enable TPU with 1 core or 8 cores
    'cpu_workers': os.cpu_count(), #questionとcontextを合わせた最大語数。今回のデータセットは1300は超えない
    'note':"リクルートベースライン",
}

args

#seed値を固定
def set_seed(seed =42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
set_seed(seed=args["random_seed"])

In [ ]:
datasetdict = datasets.load_dataset(
    args['dataset'], use_auth_token=args['huggingface_auth_token'])
datasetdict = datasetdict.flatten()\
            .rename_column('answers.text', 'answer')\
            .rename_column('answers.answer_start', 'answer_start')\
            .rename_column('answers.answer_type', 'answer_type')
train =pd.DataFrame(datasetdict['train'][:].values(), index=datasetdict['train'][:].keys()).T
train.head()

In [ ]:
sample_context= train["context"][100:105]
#sample_context= [train["context"][0]]
sample_context

In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
BertJapanese_tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
T5Tokenizer_tokenizer=T5Tokenizer.from_pretrained('rinna/japanese-roberta-base',model_max_length=384)
electra_tokenizer = AutoTokenizer.from_pretrained("Cinnamon/electra-small-japanese-generator")
electra_tokenizer = AutoTokenizer.from_pretrained("Cinnamon/electra-small-japanese-generator")

In [ ]:
def tokneize_QA(question,text,Tokenizer= AutoTokenizer,pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking"):
    Tokenizer = Tokenizer.from_pretrained(pretrained_tokenizer)
    input_ids = Tokenizer.encode(question, text)
    tokens = Tokenizer.convert_ids_to_tokens(input_ids)
    return tokens

In [ ]:
def tokenizer(pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking",Tokenizer= AutoTokenizer,contexts= sample_context):
    Tokenizer = Tokenizer.from_pretrained(pretrained_tokenizer)
    for i in range(len(contexts)):
        print("オリジナル")
        print(contexts[i])
        print(pretrained_tokenizer)
        encoding = Tokenizer.encode(contexts[i])
        tokens = Tokenizer.convert_ids_to_tokens(encoding)
        print(tokens)
        print("------------------------------------------------------------------------")
    

In [ ]:
tokenizer(pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking",Tokenizer= AutoTokenizer)
tokenizer(pretrained_tokenizer = "rinna/japanese-roberta-base",Tokenizer= AutoTokenizer)
# tokenizer(pretrained_tokenizer = "rinna/japanese-roberta-base",Tokenizer= BertJapaneseTokenizer)
tokenizer(pretrained_tokenizer = "rinna/japanese-roberta-base",Tokenizer= T5Tokenizer)
# tokenizer(pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking",Tokenizer= AutoTokenizer)

In [ ]:
#公式サイトで指示されている通りに使い分ける。
tokenizer = AutoTokenizer.from_pretrained(
            args["pretrained_tokenizer"]
            if args["pretrained_tokenizer"] else
            args["pretrained_model"])

pad_on_right = tokenizer.padding_side == "right"
#detasetdictをbertに入る形に加工
def preprocess_function(examples):
    #長文を入れた時に、二つに分ける処理は未実装
    tokenized_examples = tokenizer(
        examples['question' if pad_on_right else "context"],
        examples['context' if pad_on_right else "question"],
        return_overflowing_tokens=True, 
        padding="max_length",
    )

    inputs = {
        'input_ids': [],
        'attention_mask': [],
        'start_positions': [],
        'end_positions': [],
    }
    
    for tokens, att_mask, type_ids, context, answer,question,start_char \
            in zip(tokenized_examples['input_ids'],
                    tokenized_examples['attention_mask'],
                    tokenized_examples['token_type_ids'],
                    examples['context'],
                    examples['answer'],
                    examples['question'],
                    examples['answer_start']):
                
        #答えの場所をトークンで測るとどこになるかを計算
        answer = answer[0]
        start_char = start_char[0]
        offsets = get_offsets(tokens, context, tokenizer,
                                args["norm_form"])
        ctx_start = tokens.index(tokenizer.sep_token_id) + 1
        answer_start_index = 0
        answer_end_index = len(offsets) - 2
        while offsets[answer_start_index][0] < start_char:
            answer_start_index += 1
        while offsets[answer_end_index][1] > start_char + len(answer):
            answer_end_index -= 1
        answer_start_index += ctx_start
        answer_end_index += ctx_start

        span_inputs = {
            'input_ids': tokens,
            'attention_mask': att_mask,
            'token_type_ids': type_ids,
        }
        for span, answer_idx in make_spans(
                span_inputs,
                question_len=ctx_start,
                max_seq_len=args["max_length"],
                stride=args["doc_stride"],
                answer_start_position=answer_start_index,
                answer_end_position=answer_end_index):
            inputs['input_ids'].append(span['input_ids'])
            inputs['attention_mask'].append(span['attention_mask'])
            inputs['start_positions'].append(answer_idx[0])
            inputs['end_positions'].append(answer_idx[1])
    return inputs


def make_spans(
    inputs: Dict[str, Union[int, List[int]]],
    question_len: int,
    max_seq_len: int,
    stride: int,
    answer_start_position: int = -1,
    answer_end_position: int = -1
) -> Iterator[Tuple[Dict[str, List[int]], Tuple[int, int]]]:
    input_len = len(inputs['input_ids'])
    context_len = input_len - question_len

    def make_value(input_list, i, padding=0):
        context_end = min(max_seq_len - question_len, context_len - i)
        pad_len = max_seq_len - question_len - context_end
        val = input_list[:question_len]
        val += input_list[question_len + i:question_len + i + context_end]
        val[-1] = input_list[-1]
        val += [padding] * pad_len
        return val

    for i in range(0, input_len - max_seq_len + stride, stride):
        span = {key: make_value(val, i) for key, val in inputs.items()}
        answer_start = answer_start_position - i
        answer_end = answer_end_position - i
        if answer_start < question_len or answer_end >= max_seq_len - 1:
            answer_start = answer_end = 0
        yield span, (answer_start, answer_end)
        

def get_offsets(input_ids: List[int],
                context: str,
                tokenizer: AutoTokenizer,
                norm_form='NFKC') -> List[Tuple[int, int]]:
    
    cxt_start = input_ids.index(tokenizer.sep_token_id) + 1
    cxt_end = cxt_start + input_ids[cxt_start:].index(tokenizer.sep_token_id)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[cxt_start:cxt_end])
    tokens = [tok[2:] if tok.startswith('##') else tok for tok in tokens]
    whitespace = string.whitespace + '\u3000'

    # 1 . Make offsets of normalized context within the original context.
    offsets_norm_context = []
    norm_context = ''
    for idx, char in enumerate(context):
        norm_char = unicodedata.normalize(norm_form, char)
        norm_context += norm_char
        offsets_norm_context.extend([idx] * len(norm_char))
    norm_context_org = unicodedata.normalize(norm_form, context)
    assert norm_context == norm_context_org, \
        'Normalized contexts are not the same: ' \
        + f'{norm_context} != {norm_context_org}'
    assert len(norm_context) == len(offsets_norm_context), \
        'Normalized contexts have different numbers of tokens: ' \
        + f'{len(norm_context)} != {len(offsets_norm_context)}'
    
    offsets_token = []
    unk_pointer = None
    cid = 0
    tid = 0
    while tid < len(tokens):
        cur_token = tokens[tid]
        if cur_token == tokenizer.unk_token:
            unk_pointer = tid
            offsets_token.append([cid, cid])
            cid += 1
        elif norm_context[cid:cid + len(cur_token)] != cur_token:
            assert unk_pointer is not None, \
                'Normalized context and tokens are not matched'
            prev_unk_expected = offsets_token[unk_pointer]
            prev_unk_expected[1] += norm_context[prev_unk_expected[1] + 2:]\
                .index(tokens[unk_pointer + 1]) + 1
            tid = unk_pointer
            offsets_token = offsets_token[:tid] + [prev_unk_expected]
            cid = prev_unk_expected[1] + 1
        else:
            start_pos = norm_context[cid:].index(cur_token)
            if start_pos > 0 and tokens[tid - 1] == tokenizer.unk_token:
                offsets_token[-1][1] += start_pos
                cid += start_pos
                start_pos = 0
            assert start_pos == 0, f'{start_pos} != 0 (cur: {cur_token}'
            offsets_token.append([cid, cid + len(cur_token) - 1])
            cid += len(cur_token)
            while cid < len(norm_context) and norm_context[cid] in whitespace:
                offsets_token[-1][1] += 1
                cid += 1
        tid += 1
    if tokens[-1] == tokenizer.unk_token:
        offsets_token[-1][1] = len(norm_context) - 1
    else:
        ##長文を折り返すとここでエラーが出る。
        assert cid == len(norm_context) == offsets_token[-1][1] + 1, \
            'Offsets do not include all characters'
    assert len(offsets_token) == len(tokens), \
        'The numbers of tokens and offsets are different'

    offsets_mapping = [(offsets_norm_context[start], offsets_norm_context[end])
                       for start, end in offsets_token]
    return [(0, 0)] + offsets_mapping+[(0, 0)] 

In [ ]:
#テスト用
features = preprocess_function(datasetdict['train'][100:105])
#tokenized_datasets = datasetdict.map(preprocess_function, batched=True, remove_columns=datasetdict["train"].column_names)

In [ ]:
features

In [ ]:
tokenizer(pretrained_tokenizer = "cl-tohoku/bert-base-japanese-whole-word-masking",Tokenizer= AutoTokenizer)

In [ ]:
tokens= tokneize_QA(train["question"][102],train["context"][102])

In [ ]:
print(tokens)


In [ ]:
tokens[21:28]
#  'start_positions': [24, 0, 48, 0, 21, 0, 52, 0, 36, 0],
#  'end_positions': [50, 0, 52, 0, 28, 0, 53, 0, 41, 0]}